In [1]:
import csv

In [14]:
with open('./data/converted_crime_data/Crime_pass5_no_tracts.csv') as f:
    data = [r for r in csv.reader(f)]
with open('./data/converted_crime_data/Crime_pass6_cases.csv') as f:
    cases = [r for r in csv.reader(f)]
with open('./data/converted_crime_data/Crime_pass6_codes.csv') as f:
    codes = [r for r in csv.reader(f)]
with open('./data/converted_crime_data/Crime_pass6_locations.csv') as f:
    locations = [r for r in csv.reader(f)]

In [6]:
from itertools import zip_longest
list(enumerate(zip_longest(data[0], locations[0])))

[(0, ('address', 'address')),
 (1, ('apt', 'fid')),
 (2, ('case', 'building_name')),
 (3, ('primary_offense', 'x')),
 (4, ('offense_code', 'y')),
 (5, ('offense', None)),
 (6, ('date_rept', None)),
 (7, ('date_occu', None)),
 (8, ('hour_occu', None)),
 (9, ('date_fnd', None)),
 (10, ('hour_fnd', None)),
 (11, ('fid', None)),
 (12, ('x', None)),
 (13, ('y', None))]

In [10]:
(len(data[1:]), sum(r[0] == '' for r in data[1:]))

(8137, 7)

In [18]:
data_with_location = [data[0]] + [
    r for r in data[1:] if not (r[0] == '' and r[11] == '' and r[12] == ''  and r[13] == '')]

In [19]:
data_with_location[:2]

[['address',
  'apt',
  'case',
  'primary_offense',
  'offense_code',
  'offense',
  'date_rept',
  'date_occu',
  'hour_occu',
  'date_fnd',
  'hour_fnd',
  'fid',
  'x',
  'y'],
 ['1525 ALBENGA AV',
  '',
  '07000101',
  '1',
  'VAB',
  'VANDALISM TO VEHICLE',
  '2007-01-04',
  '2007-01-03',
  '1600',
  '2007-01-04',
  '1600',
  '109',
  '891786.45900000003',
  '502485.90500000003']]

In [20]:
sum(r[4] == '' for r in data_with_location)

0

so we don't need codes or cases csv files at this point

In [100]:
sum((r[12] == '' or r[13] == '' or r[11] == '') and r[0] != '' for r in data_with_location)

4583

so the existence of fid, x, or y are independent

In [41]:
import re
rex1 = re.compile(r'\W+')
rex2 = re.compile(r'\.')
clean_addr = lambda s: rex1.sub(' ', rex2.sub(' ', s)).upper().strip()

In [63]:
addresses = {clean_addr(r[0]): r[1:] for r in locations if clean_addr(r[0]) != ''}

In [64]:
fixables = [r for r in data[1:] if r[11] == '' and r[0] in addresses]

In [97]:
data_fix_missing_locations = [data_with_location[0]]
for row in data_with_location[1:]:
    if (row[11] == '' or row[12] == '') and row[0] in addresses:
        fid, apt, x, y = addresses[row[0]]
        data_fix_missing_locations.append([row[0]] + [clean_addr(apt)] + row[2:11] + [fid, x, y])
    else:
        data_fix_missing_locations.append(row)

In [98]:
need_to_geolocate = [r for r in data_fix_missing_locations
                    if r[12] == '' and r[13] == '' and r[0] != '']

In [108]:
[len(need_to_geolocate), len(data_fix_missing_locations)]

[3457, 8131]

In [89]:
need_to_geolocate[0]

['1231 THEO DICKINSON DR',
 '',
 '07009517',
 '1',
 'ALA',
 'ALCOHOL JUVENILE VIOLATION',
 '2007-10-21',
 '2007-10-21',
 '0325',
 '2007-10-21',
 '0325',
 '',
 '',
 '']

In [101]:
sum(r[0] == '1101 STANFORD DR' for r in need_to_geolocate)

214

http://www.miami.edu/sa/index.php/residential_life/services/mail_packages/

* Eaton Residential College	1211 Dickinson Drive
* Mahoney Residential College	1101 Stanford Drive
* Pearson Residential College	5185 Ponce de Leon Blvd.
* Hecht Residential College 	1231 Dickinson Drive
* Stanford Residential College	1239 Dickinson Drive
* University Village	1527 Albenga Avenue

In [141]:
from collections import Counter
geolocate_counter = Counter(r[0] for r in need_to_geolocate)

top10 = sorted(geolocate_counter.items(), key=lambda x: x[1])[-10:]
print(sum(x[1] for x in top10))
list(reversed(top10))

1864


[('1231 THEO DICKINSON DR', 352),
 ('1239 THEO DICKINSON DR', 263),
 ('5665 PONCE DE LEON BLVD', 228),
 ('1101 STANFORD DR', 214),
 ('5185 PONCE DE LEON BLVD', 203),
 ('1231 DICKINSON DR', 194),
 ('1239 DICKINSON DR', 159),
 ('5501 SAN AMARO DR', 103),
 ('1211 THEO DICKINSON DR', 81),
 ('1311 MILLER DR', 67)]

among the top 10 are the flipse building/garage and the music recital building

In [151]:
[len(geolocate_counter), sum(x for x in geolocate_counter.values())]

[307, 3457]

In [131]:
from geopy.geocoders import GoogleV3

In [132]:
geolocator = GoogleV3(api_key='')

In [142]:
missing_locations = list(geolocate_counter.keys())

In [146]:
from time import sleep

In [148]:
geolocated = {}
failed = []
for loc in missing_locations:
    res = geolocator.geocode(loc + ' CORAL GABLES FL')
    if res is None:
        print('failed to geolocate {}'.format(loc))
        failed.append(loc)
    else:
        print('got {}'.format(loc))
        geolocated[loc] = res
    sleep(1 / 10) # API throttling

got 1599 CORNICHE AV/RED RD
got DICKINSON DR/PONCE DE LEON BLVD
got 5801 PONCE DE LEON BLVD
got 1549 S ALHAMBRA CIR/PONCE DE LEON BLVD
got 1100 S DIXIE HWY
got 1500 ALBENGA AV/SAN AMARO DR
got 5300 PONCE DE LEON BLVD
got 1225 ALBENGA AVE
got 1211 DICKINSON DR
got 1531 LIGURIA AV
got 5600 PONCE DE LEON BLVD
got 1305 CAMPO SANO AV
got 5199 PONCE DE LEON BLVD/STANFORD DR
got 1111 MEMORIAL DRI
got 5500 BLK SAN AMARO DR
got 4900 CAMPO SANO CT
got 5616 GEORGE E MERRICK ST
got 5100 PONCE DE LEON BLVD
got SAN AMARO DR/PONCE DE LEON BLVD
got 5300 CARILLO ST
got 5600 GRANADA BLVD
got 1561 CORNICHE AV
got 5600 GEORGE E MERRICK ST
got RED RD/CORNICHE AV
got SAN AMARO DR/LEVANTE AV
got RED RD/LEVANTE AV
got 6501 SAN AMARO DR
got 1575 LIGURIA DR
got 1400 S DIXIE HWY
got 5600 RED RD
got SAN AMARO DR/ROBBIA AV
got 5225 STANFORD DR
got 5616 PAVIA ST
got 5299 PONCE DE LEON BLVD/GEORGE E MERRICK ST
got 5801 SAN AMARO DR
got 1299 DICKINSON DR
got 1537 LIGURIA AV
got 1231 THEO DICKINSON DR
got PONCE DE LEO

In [150]:
len(failed)

0

In [153]:
res = list(geolocated.keys())[0]

In [157]:
[x for x in dir(geolocated[res]) if not x.startswith('_')]

['address', 'altitude', 'latitude', 'longitude', 'point', 'raw']

In [161]:
geolocated[res].latitude

25.7140872

In [166]:
with open('./data/cached_geocode_data.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['address', 'geolocated_address', 'lat', 'lon', 'altitude'])
    for key, value in geolocated.items():
        writer.writerow([key, value.address, value.latitude, value.longitude, value.altitude])